In [2]:
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import translators as ts
import json
from selenium.webdriver.chrome.options import Options

d:\Anaconda3\envs\spider\Lib\site-packages\translators\apis.py:162: UserWarning: Unable to find server backend.

  warnings.warn('Unable to find server backend.\n')
Using  server backend.


获取摘要和翻译,关键词

In [3]:
url="https://ieeexplore.ieee.org/document/9411225"
url="https://ieeexplore.ieee.org/document/7809977" #会议
url="https://ieeexplore.ieee.org/document/5165285" #TWC
html=urlopen(url)
html_byte=html.read()

In [4]:
f=open(".\\html\\get detail test.html","wb")
f.write(html_byte)
f.close()

In [5]:
bs=BeautifulSoup(html_byte)
abstract = bs.find('meta', {'property': 'twitter:description'})['content']
print(abstract)

abstract_translation=ts.google(abstract,from_language="en",to_language='zh', host_url="https://translate.google.com/")
print(abstract_translation)

In this letter it is shown that by taking advantage of the particle velocity, in addition to the pressure, multichannel reception can be accomplished in underwater acoustic wireless channels. Theoretical formulation and Monte Carlo simulations are provided for a vector sensor equalizer that measures the pressure and the velocity at a single point in space. These results demonstrate the usefulness of small-size vector sensors as multichannel equalizers for underwater acoustic wireless systems and sensor networks.
在这封信中，表明，通过利用粒子速度，除了压力外，多通道接收可以在水下声学无线通道中完成。为矢量传感器均衡器提供了理论公式和蒙特卡洛模拟，该矢量传感器均衡器测量了空间中一个点的压力和速度。这些结果证明了小型矢量传感器作为水下声学无线系统和传感器网络的多通道均衡器的有用性。


关键词获取 keywords

In [6]:
temp=bs.find("body",{"class","body-resp"}).find('div', id="LayoutWrapper").find('div', {"class",'col'}).find_all("script", type="text/javascript")

for script_tag in temp:
    if script_tag:
        script_text=script_tag.string
        if script_text:
            match = re.search(r'xplGlobal\.document\.metadata\s*=\s*(\{.*?\};)', script_text, re.DOTALL) #正则表达式匹配
            if match:
                metadata_text = match.group()
                print(metadata_text)
                break
        else:
            continue
    else:
        continue

xplGlobal.document.metadata={"userInfo":{"customerNameRaw":"SHENZHEN UNIVERSITY","institutionName":"SHENZHEN UNIVERSITY","institute":true,"member":false,"individual":false,"guest":false,"subscribedContent":false,"fileCabinetContent":false,"fileCabinetUser":false,"institutionalFileCabinetUser":false,"products":"IEL|VDE|NOKIA BELL LABS|","instType":"Academic","userIds":[12631],"showPatentCitations":true,"showGet802Link":true,"openUrlImgLoc":"/assets/img/btn.find-in-library.png","openUrlLink":"NA","showOpenUrlLink":false,"marketingInfoCaptured":false,"tracked":false,"ringGoldId":"47890","delegatedAdmin":false,"desktop":false,"isInstitutionDashboardEnabled":false,"isInstitutionProfileEnabled":false,"isRoamingEnabled":true,"isDelegatedAdmin":false,"isMdl":false,"isCwg":false,"isIel":true,"isAcademic":true},"authors":[{"name":"Ali Abdi","affiliation":["Center for Wireless Communications and Signal Processing Research, Department of Electrical and Computer Engineering, New Jersey Institute of

In [7]:
keywords_string=re.search(r'"keywords":\s*\[.*?\],"', metadata_text, re.DOTALL).group()

All_Keywords_list_string=re.search(r'\[.*\]', keywords_string, re.DOTALL)

keywords_list=json.loads(All_Keywords_list_string.group())[3]['kwd']
print(keywords_list)

['Multichannel equalization, particle velocity, underwater communication, vector sensors.']


寻找参考文献的链接, 依赖html_driver

In [8]:
driver = webdriver.Chrome(executable_path=".\\chromedriver.exe")
driver.get(url)
time.sleep(10)
html = driver.page_source
driver.quit()

C:\Users\Anable\AppData\Local\Temp\ipykernel_9380\1474514747.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=".\\chromedriver.exe")


In [9]:
f=open(".\\html\\get detail test2.html","w")
f.write(html)
f.close()

In [10]:
bs_driver=BeautifulSoup(html)
View_Article=bs_driver.find_all("a",{"class":'stats-reference-link-viewArticle'})
CrossRef=bs_driver.find_all("a",{"class":'stats-reference-link-crossRef'})
View_Article_list=[]
CrossRef_list=[]

for article in View_Article:
    if article:
        View_Article_list.append(article.get("href"))
        
View_Article_list=['https://ieeexplore.ieee.org'+link for link in View_Article_list]

for article in CrossRef:
    if article:
        CrossRef_list.append(article.get("href"))

print(View_Article_list,CrossRef_list)

['https://ieeexplore.ieee.org/document/4449191', 'https://ieeexplore.ieee.org/document/1637927', 'https://ieeexplore.ieee.org/document/4610779'] ['https://doi.org/10.1121/1.1914426', 'https://doi.org/10.1007/978-3-662-11761-3']


获取所在的期刊名

In [11]:
Journal_Title=bs_driver.find('strong', text='Published in: ').next_sibling.get_text()
print(Journal_Title)

'''impact_factor_list={'IEEE Transactions on Wireless Communications':10}
if Journal_Title in impact_factor_list:
    impact_factor=str(impact_factor_list[Journal_Title])
else:
    impact_factor='null'

print(impact_factor)'''


IEEE Transactions on Wireless Communications


"impact_factor_list={'IEEE Transactions on Wireless Communications':10}\nif Journal_Title in impact_factor_list:\n    impact_factor=str(impact_factor_list[Journal_Title])\nelse:\n    impact_factor='null'\n\nprint(impact_factor)"

获取DOI

In [12]:
DOI='https://doi.org/'+bs_driver.find('strong', text='DOI: ').next_sibling.get_text()
print(DOI)

https://doi.org/10.1109/TWC.2009.070566


获取影响因子 只在期刊和杂志中才有

In [13]:
Journal_index=False

if bs_driver.find(text='Journals & Magazines'):

    Journal_index=True 
    print('this is a Journals or Magazines paper')

    journal_link='https://ieeexplore.ieee.org'+bs_driver.find('strong', text='Published in: ').next_sibling.get('href')
    print(journal_link)

    driver = webdriver.Chrome(executable_path=".\\chromedriver.exe")
    driver.get(journal_link)
    time.sleep(5)
    journal_html = driver.page_source
    driver.quit()

    f=open(".\\html\\get detail test3.html","w")
    f.write(journal_html)
    f.close()

    bs_temp=BeautifulSoup(journal_html)
    impact_factor=bs_temp.find('a',{'class':'stats-jhp-impact-factor'}).find('span',{'class':'text-md-md-lh'}).get_text()
    
else :
    impact_factor='null'

print(impact_factor)

this is a Journals or Magazines paper
https://ieeexplore.ieee.org/xpl/RecentIssue.jsp?punumber=7693


C:\Users\Anable\AppData\Local\Temp\ipykernel_9380\3533504792.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=".\\chromedriver.exe")


8.346


获取标题 title

In [14]:
print(bs.find('title'))

<title>A new compact multichannel receiver for underwater wireless communication networks | IEEE Journals &amp; Magazine | IEEE Xplore</title>


In [15]:
class Article:
    #公共成员
    Main_Index=False
    Journal_Index=False

    Html='' 
    Url=''
    
    Title='' #依赖html?
    Date='' #依赖html?
    Abstract=''  #依赖 Html
    Abstract_Translation=''

    #主要成员
    Keywords=[]    #依赖html
    References_Link=[] #依赖html_driver

    #参考成员
    Journal_Name='' #依赖html?
    Html_Driver=''
    Impact_Factor='' #依赖html_driver
    Doi=''

    def __init__(self, url='', main_index=False, **kwargs):

        self.Url=url
        self.Main_Index=main_index
        
        '''        
        for key, wd in kwargs:
            
            if key =='Title' or 'title':
                self.Title=wd
            if key =='Keywords' or 'Keyword':
                self.Keywords=wd
            if key =='Abstract' or 'abstract':
                self.Abstract=wd
        '''

        if url:
            '''            
            self.get_html()
            if not self.Main_Index:
                self.get_html_driver()
            '''

            self.get_html()
            self.get_html_driver()

            if self.Main_Index:

                self.get_title()
                self.get_date()
                self.get_abstract()
                self.get_keywords()
                self.get_references()
                Keywords=[]    #依赖html
                References_Link=[] #依赖html_driver

            
        '''
        if self.Html:
            self.get_abstract()
            self.get_title()
            self.get_date()

        if self.Html_driver:
            self.get_journal_name()
            self.get_abstract()
            self.get_title()
        '''

    def get_html(self):
        self.Html=urlopen(url).read().decode('utf-8')
        return 0
    
    def get_html_driver(self):
        driver = webdriver.Chrome(executable_path=".\\chromedriver.exe")
        driver.get(url)
        time.sleep(10)
        self.Html_driver = driver.page_source
        driver.quit()

    def get_abstract(self):
        #如果发生异常,需要修改包'translators'的源码,将链接translate.google.cn改为.com
        if self.Abstract:
            return 0
        
        if self.Html:
            bs=BeautifulSoup(html_byte)
            self.Abstract = bs.find('meta', {'property': 'twitter:description'})['content']
            self.Abstract_Translation=ts.google(self.Abstract,from_language="en",to_language='zh', host_url="https://translate.google.com/")
        else:
            if self.Url:
                self.get_html()
                self.get_abstract()
                return 0
            else:
                print('获取摘要失败,未提供链接')
        return 0

    def get_date(self):
        if self.Date or (not self.Journal_Index): #非杂志或期刊不用获取日期
            return 0
        
        if self.Html_Driver:
            #TODO: 获取日期代码
            return 0
        
        else:
            if self.Url:
                self.get_html()
                self.get_date()
                return 0
            else:
                print('获取日期失败,未提供链接')

    def get_title(self):
        if self.Title:
            return 0
        
        if self.Html_Driver:
            #TODO: 获取标题代码
            return 0
        else:
            if self.Url:
                self.get_html()
                self.get_title()
                return 0

    def get_doi(self):
        if self.Doi or self.Main_Index: #主期刊不获取doi
            return 0
        
        if self.Html_Driver:
            #TODO: 获取日期代码
            return 0
        
        else:
            if self.Url:
                self.get_html()
                self.get_date()
                return 0
            else:
                print('获取日期失败,未提供链接')
    def get_journal_name(self):
        ok= False
        if ok:
            self.Journal_Index=True
        return 0
    
    def get_keywords(self):

        bs=BeautifulSoup(self.Html)
        temp=bs.find("body",{"class","body-resp"}).find('div', id="LayoutWrapper").find('div', {"class",'col'}).find_all("script", type="text/javascript")

        for script_tag in temp:
            if script_tag:
                script_text=script_tag.string
                if script_text:
                    match = re.search(r'xplGlobal\.document\.metadata\s*=\s*(\{.*?\};)', script_text, re.DOTALL) #正则表达式匹配
                    if match:
                        metadata_text = match.group()
                        break
                else:
                    continue
            else:
                continue
        
        keywords_string=re.search(r'"keywords":\s*\[.*?\],"', metadata_text, re.DOTALL).group()
        All_Keywords_list_string=re.search(r'\[.*\]', keywords_string, re.DOTALL)
        self.Keywords=json.loads(All_Keywords_list_string.group())[3]['kwd']

        return 0

    def get_references(self):
        return 0        
    
    def get_pdf(self,save_path='.\\download Pdf'):
        urlopen('https://scihub.se/'+self.Doi)

    